## Importing libraries

In [9]:
import sklearn.tree as tree
import pandas as pd
from hazm import Stemmer ,Lemmatizer

## Open CorpusText & StopWords 

In [10]:

corpus_original = pd.read_csv('./FarsEconomyAllCleaned.csv', sep='\t',index_col='Unnamed: 0')
corpus = corpus_original.copy()
stopWords = pd.read_csv('./stopWords.txt', sep='\t',header=None)
stopWords = list(stopWords[0])
stopWords.append('تر')
stopWords.append('ترین')
stopWords.append('ها')
stopWords.append('های')
stopWords.append('ای')

# stopWords.head()
# corpus.head()

## Get text data 

In [11]:
corpus = corpus.drop(['date'],axis=1)
corpus = corpus[['title','summery','body']]
corpus.head()

,title,summery,body
0,بررسی تخلفات واگذاری آلومینیوم المهدی هفته آین...,عضو کمیسیون انرژی مجلس گفت: تخلفات مربوط به جر...,\nاحمد مرادی نماینده مجلس شورای اسلامی در گفت‌...
1,پربیننده‌ترین اخبار خبرگزاری فارس در 24ساعت گذ...,غش کردن مردمی که دلار خریده بودند! و سه‌گانه ا...,\nپربیننده‌ترین اخبار خبرگزاری فارس در روز گذش...
2,شرکت هندی برای جایگزینی نفت ایران به نفوذ شرکا...,نارایا انرژی، یکی از خریداران هندی و بزرگ نفت ...,\nبه گزارش گروه اقتصاد بین‌الملل فارس به نقل ا...
3,سیر نزولی قیمت خودرو با سقوط نرخ دلار آغاز شد,قیمت انواع خودروهای داخلی و خارجی امروز در باز...,\nبه گزارش خبرنگار اقتصادی خبرگزاری فارس، قیمت...
4,دلایل تاخیر در تحویل خودرو‌ها/ چرا خودروهای پر...,قائم مقام مرکز مطالعات استراتژیک فناوری اطلاعا...,\nبه گزارش خبرنگار اقتصادی خبرگزاری فارس، بعد ...


## Let's Start

We want model our document to vector

We find 100 important words from our corpus

at first we set weight:
- words in title: 3x 
- words in summary: 2x 
- words in body: 1x 

### 1- set vector dimention

some cleaninc in text for some chars: '\n' , ':' , '.' , ',' and ...

In [12]:
corpus = corpus_original.copy()
def clean(text):
    text = text.replace('\n',' ').\
                replace(':','').\
                replace('.','').\
                replace('،','').\
                replace('/','').\
                replace('فیلم','').\
                replace('+','').\
                replace('»','').\
                replace('«','').\
                replace('(','').\
                replace(')','').\
                replace('\xa0','').\
                replace('\u200c',' ')
    
    text = text.replace('گزارش','').\
                replace('فارس','').\
                replace('نقل','').\
                replace('خبرگزاری','').\
                replace('خبرنگار','')
    
    
    text = text.replace('  ',' ')
    t = text.split(' ')
    t = [x for x in t if x not in stopWords and\
                           not x.isdigit() and\
                           not '']
    return t

corpus['title']   = corpus['title'].apply(clean)
corpus['summery'] = corpus['summery'].apply(clean)
corpus['body']    = corpus['body'].apply(clean)


In [14]:
dictionary = {}

#inner function
stem = lambda x:Stemmer().stem(x)
lem = lambda x:Lemmatizer().lemmatize(x)
def cbow_adder(x,importance=1):
    if x in dictionary.keys():
        dictionary[x] += 1*importance
    else:
        dictionary[x] = 1*importance
    return x

#outter function
cbow_maker     = lambda txt: (list(map(cbow_adder,txt)))
cbow_stemmer   = lambda txt: (list(map(stem,txt)))
cbow_lemmatize = lambda txt: (list(map(lem,txt)))

#start function
# corpus.title   = corpus.title.apply(cbow_lemmatize)
# print('       ','lemma done!')
# corpus.title   = corpus.title.apply(cbow_stemmer)
# print('       ','stm done!')
corpus.title   = corpus.title.apply(cbow_maker)
# print('       ','maker done!')

dictionary = {k:v*1.5 for k, v in dictionary.items()}

# corpus.summery   = corpus.summery.apply(cbow_lemmatize)
# corpus.summery   = corpus.summery.apply(cbow_stemmer)
corpus.summery   = corpus.summery.apply(cbow_maker)
dictionary = {k:v*2 for k, v in dictionary.items()}

# corpus.body   = corpus.body.apply(cbow_lemmatize)
# corpus.body   = corpus.body.apply(cbow_stemmer)
corpus.body   = corpus.body.apply(cbow_maker)

In [17]:
dictionary
di = {
    'word':[*dictionary],
    'frequency':list(dictionary.values())
}
cbow = pd.DataFrame.from_dict(di)
cbow = cbow.sort_values(by='frequency',ascending=False)
cbow = cbow[cbow.word != '']
dims = cbow.iloc[:100]
# dictionary.clear()
# dictionary
dims.to_csv('./VectorDim.csv')

In [456]:
stm = pd.read_csv('./PTB_stem_dataset.txt', sep='\t',header=None)
stm[stm[0]== 'آمریکا']


,0,1,2
546,آمریکا,آمریکا,Nas


In [457]:
lem = pd.read_csv('./PDTB_lemma_dataset.txt', sep='\t',header=None)
lem[lem[1] == 'آمریکا']

,0,1,2
3068,آمریکا,آمریکا,N
3069,آمریکای,آمریکا,N


In [464]:
>>> from hazm import Stemmer, Lemmatizer
>>> stemmer = Stemmer()
>>> stemmer.stem('کتاب‌ها')
>>> lemmatizer = Lemmatizer()
>>> lemmatizer.lemmatize('می‌روم')

'رفت#رو'

In [18]:
# corpus.title